<a href="https://colab.research.google.com/github/abinavharsath41-ctrl/FOML-exp/blob/main/Another_copy_of_multilayer_perceptron.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 # Step 1: Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import requests
from io import StringIO

# --- Configuration based on Exercise 5 ---
RANDOM_SEED = 42
MAX_ITER = 500
VALIDATION_FRACTION = 0.1
ARCHITECTURES = [(10, 10)] # Two hidden layers, 10 neurons each
SOLVER = 'adam'


def load_banknote_data():
    """
    Downloads the UCI Banknote Authentication dataset from the UCI repository.
    """
    print("Loading Banknote Authentication dataset...")
    # Direct link to the raw data file (common for UCI datasets)
    data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/00267/data_banknote_authentication.txt"
    try:
        response = requests.get(data_url, timeout=10)
        response.raise_for_status() # Raise an HTTPError for bad responses

        # Define column names based on the UCI documentation
        col_names = [
            'variance_of_wavelet',
            'skewness_of_wavelet',
            'curtosis_of_wavelet',
            'entropy_of_image',
            'class' # The target variable
        ]

        # Load data directly from the text content
        data = pd.read_csv(StringIO(response.text), header=None, names=col_names)
        print("Data loaded successfully.")
        return data
    except requests.exceptions.RequestException as e:
        print(f"Error downloading data. Please check the URL or your internet connection: {e}")
        # Fallback: Create a dummy DataFrame to prevent script crash
        return pd.DataFrame()


def build_train_and_evaluate_mlp(X_train, X_test, y_train, y_test, activation_func, architecture, max_iter):
    """
    Builds, trains, and evaluates an MLPClassifier with given parameters.
    """
    print(f"\n--- Running for Activation: {activation_func} ---")

    # Step 6: Build MLP model
    # Note: validation_fraction is handled internally by scikit-learn's MLPClassifier
    mlp = MLPClassifier(
        hidden_layer_sizes=architecture,
        activation=activation_func, # Step 6c: Activation to test
        solver=SOLVER,             # Step 6b: Optimizer
        alpha=0.0001,              # L2 penalty (regularization term parameter)
        max_iter=max_iter,         # Step 6d: Learning controls
        random_state=RANDOM_SEED,  # Step 6d: Random seed
        early_stopping=True,       # Step 6d: Early stopping enabled
        validation_fraction=VALIDATION_FRACTION, # Step 6d: Validation fraction
        n_iter_no_change=10,       # Stop if loss doesn't improve for 10 consecutive epochs
        verbose=False              # Set to True to see training progress
    )

    # Step 7: Train the model
    print(f"Training model (Architecture: {architecture}, Max Iter: {max_iter})...")

    # The fit method records the training progress in the 'loss_curve_' and 'validation_scores_' attributes
    mlp.fit(X_train, y_train)

    # Step 8: Do the testing process and compute metrics
    y_pred = mlp.predict(X_test)

    # Compute Confusion Matrix (TN, FP, FN, TP)
    cm = confusion_matrix(y_test, y_pred)
    # cm is [[TN, FP], [FN, TP]] for a binary classification
    TN, FP, FN, TP = cm.ravel()

    # Compute accuracy, precision, recall, f1-score for each class
    accuracy = accuracy_score(y_test, y_pred)
    # precision_recall_fscore_support returns (precision, recall, f1, support)
    precision, recall, f1, _ = precision_recall_fscore_support(y_test, y_pred, average=None)

    print("\nEvaluation Metrics:")
    print(f"  Confusion Matrix:\n{cm}")
    print(f"  TN={TN}, FP={FP}, FN={FN}, TP={TP}")
    print(f"  Accuracy: {accuracy:.4f}")
    print(f"  Precision (Class 0, Class 1): {precision}")
    print(f"  Recall    (Class 0, Class 1): {recall}")
    print(f"  F1-Score  (Class 0, Class 1): {f1}")

    return mlp

def plot_training_curves(mlp, activation_func):
    """
    Step 9: Plot training loss and validation accuracy per epoch.
    """

    if len(mlp.loss_curve_) > 0:
        epochs = range(1, len(mlp.loss_curve_) + 1)

        # Plot Loss
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        plt.plot(epochs, mlp.loss_curve_, label='Training Loss')
        plt.title(f'Training Loss per Epoch ({activation_func})')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.grid(True)
        plt.legend()

        # Plot Validation Accuracy (Note: validation_scores_ is only available when early_stopping=True)
        if len(mlp.validation_scores_) > 0:
            plt.subplot(1, 2, 2)
            plt.plot(epochs[:len(mlp.validation_scores_)], mlp.validation_scores_, label='Validation Accuracy', color='orange')
            plt.title(f'Validation Accuracy per Epoch ({activation_func})')
            plt.xlabel('Epochs')
            plt.ylabel('Accuracy')
            plt.grid(True)
            plt.legend()

        plt.tight_layout()
        plt.show()
    else:
        print("Could not retrieve loss curve data.")

def main():
    # Load the data
    data = load_banknote_data()

    if data.empty:
        print("Cannot proceed without data.")
        return

    # Step 3: View first few rows of the dataset
    print("\nFirst 5 rows of the dataset:")
    print(data.head())
    print("\nDataset Info:")
    data.info()

    # Step 4: Separate features (X) and target (y)
    X = data.iloc[:, :-1]  # All columns except the last one (features)
    y = data.iloc[:, -1]   # The last column (target)

    # Step 5: Split the dataset for training and testing
    # Using stratify=y ensures that the train/test split has the same proportion of class labels
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=RANDOM_SEED, stratify=y
    )
    print(f"\nData split into training ({len(X_train)} samples) and testing ({len(X_test)} samples).")

    # Step 10: Repeat steps 6 to 9 for different activation functions
    # Tanh, Logistic (Sigmoid), Identity
    activation_functions = ['tanh', 'logistic', 'identity']

    results = {}

    for activation in activation_functions:
        # Step 6-9 execution
        mlp_model = build_train_and_evaluate_mlp(
            X_train, X_test, y_train, y_test, activation, ARCHITECTURES[0], MAX_ITER
        )
        plot_training_curves(mlp_model, activation)
        results[activation] = mlp_model


if __name__ == "__main__":
    # Ensure matplotlib is set to non-interactive mode if running in a headless environment
    try:
        plt.switch_backend('Agg')
    except ImportError:
        pass

    main()

Loading Banknote Authentication dataset...
Data loaded successfully.

First 5 rows of the dataset:
   variance_of_wavelet  skewness_of_wavelet  curtosis_of_wavelet  \
0              3.62160               8.6661              -2.8073   
1              4.54590               8.1674              -2.4586   
2              3.86600              -2.6383               1.9242   
3              3.45660               9.5228              -4.0112   
4              0.32924              -4.4552               4.5718   

   entropy_of_image  class  
0          -0.44699      0  
1          -1.46210      0  
2           0.10645      0  
3          -3.59440      0  
4          -0.98880      0  

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1372 entries, 0 to 1371
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   variance_of_wavelet  1372 non-null   float64
 1   skewness_of_wavelet  1372 non-null   float

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



--- Running for Activation: identity ---
Training model (Architecture: (10, 10), Max Iter: 500)...

Evaluation Metrics:
  Confusion Matrix:
[[220   9]
 [ 15 168]]
  TN=220, FP=9, FN=15, TP=168
  Accuracy: 0.9417
  Precision (Class 0, Class 1): [0.93617021 0.94915254]
  Recall    (Class 0, Class 1): [0.96069869 0.91803279]
  F1-Score  (Class 0, Class 1): [0.94827586 0.93333333]
